#### Import necessary libraries for data manipulation and preprocessing

In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

1. Load the Data:
Read data from various sources such as CSV files, Excel files, and SQL databases.

In [5]:
df = pd.read_excel("Online Retail.xlsx")

2. Understand the Data:
Explore basic information about the data, including head, info, and describe.
Check for missing values in the dataset.

In [7]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [8]:
df.shape

(541909, 8)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [10]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [11]:
df.isnull().mean()

InvoiceNo      0.000000
StockCode      0.000000
Description    0.002683
Quantity       0.000000
InvoiceDate    0.000000
UnitPrice      0.000000
CustomerID     0.249267
Country        0.000000
dtype: float64

 --- 3. Clean the Data ---
   
Handle missing values by either dropping or filling them.

Remove duplicate entries in the dataset.

Correct data types as necessary.

In [13]:
df_clean = df.dropna(subset=['Description', 'CustomerID'])

In [14]:
df_clean.drop_duplicates(inplace=True)

In [15]:
df_clean["InvoiceNo"] = df_clean["InvoiceNo"].astype(str).str.strip()


In [16]:
df_clean["InvoiceNo"] = df_clean["InvoiceNo"].str.replace(r"[^0-9]", "", regex=True)


In [17]:

df_clean["InvoiceNo"] = df_clean["InvoiceNo"].replace("", np.nan)


In [18]:
df_clean["InvoiceNo"] = pd.to_numeric(df_clean["InvoiceNo"], errors="coerce").astype('Int64')


In [19]:
print(df_clean["InvoiceNo"].isna().sum(), "rows still have invalid InvoiceNo")


0 rows still have invalid InvoiceNo


In [20]:

df_clean["CustomerID"] = df_clean["CustomerID"].replace("", np.nan)


In [21]:
df_clean.dropna(subset=['CustomerID'],inplace=True)

In [22]:
df_clean["InvoiceDate"] = pd.to_datetime(df_clean["InvoiceDate"], errors='coerce')

In [23]:
df_clean["UnitPrice"] = df_clean["UnitPrice"].abs()   # convert -123 → 123

In [24]:
df_clean = df_clean[df_clean["UnitPrice"] != 0]


In [ ]:
 Transform the Data & 5. Feature Engineering 

In [47]:
# Create a new feature: 'TotalPrice'
df_clean['TotalPrice'] = df_clean['Quantity'] * df_clean['UnitPrice']

In [73]:
# For customer segmentation, it's best to aggregate data at the customer level.
# We will create a new DataFrame with one row per customer.

In [49]:
customer_df = df_clean.groupby('CustomerID').agg(
    TotalSpent=('TotalPrice', 'sum'),
    TotalItems=('Quantity', 'sum'),
    TotalTransactions=('InvoiceNo', 'nunique'),
    LastPurchaseDate=('InvoiceDate', 'max')
).reset_index()

In [53]:
# Create a 'Recency' feature
# Recency is the number of days since the last purchase
latest_date = customer_df['LastPurchaseDate'].max()
customer_df['Recency'] = (latest_date - customer_df['LastPurchaseDate']).dt.days

In [ ]:
# Feature Selection: Select the most relevant features for clustering

In [57]:
# Identify numerical features for scaling
numerical_features = ['TotalSpent', 'TotalItems', 'TotalTransactions', 'Recency']

In [59]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Normalize/Standardize the Data: K-Means is sensitive to the scale of features.
# We'll use StandardScaler to give all features equal weight.

In [61]:
# Create a preprocessing pipeline to scale numerical data
preprocessor = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [63]:
# Apply the transformations
customer_df_scaled = preprocessor.fit_transform(customer_df[numerical_features])


In [67]:
# The `customer_df_scaled` is now ready for the K-Means clustering algorithm!
print("\nData has been scaled and is ready for clustering.")
print("Shape of the scaled data:", customer_df_scaled.shape)


Data has been scaled and is ready for clustering.
Shape of the scaled data: (4371, 4)
